In [ ]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession(".cache", expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 41.0138,
    "longitude": 28.9497,
    "start_date": "2005-01-01",
    "end_date": "2021-04-07",
    "hourly": "snow_depth",
    "daily": ["temperature_2m_max", "temperature_2m_min", "rain_sum", "snowfall_sum"],
    "timezone": "auto",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_snow_depth = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left",
    )
}

hourly_data["snow_depth"] = hourly_snow_depth

hourly_dataframe = pd.DataFrame(data=hourly_data)
print(hourly_dataframe)

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_rain_sum = daily.Variables(2).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(3).ValuesAsNumpy()

daily_data = {
    "date": pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left",
    )
}

daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum

daily_dataframe = pd.DataFrame(data=daily_data)
daily_dataframe.to_csv("weather.csv", index=False)

Coordinates 41.01932907104492°N 28.887304306030273°E
Elevation 32.0 m asl
Timezone b'Europe/Istanbul' b'GMT+3'
Timezone difference to GMT+0 10800 s
                            date  snow_depth
0      2004-12-31 21:00:00+00:00         0.0
1      2004-12-31 22:00:00+00:00         0.0
2      2004-12-31 23:00:00+00:00         0.0
3      2005-01-01 00:00:00+00:00         0.0
4      2005-01-01 01:00:00+00:00         0.0
...                          ...         ...
142579 2021-04-07 16:00:00+00:00         0.0
142580 2021-04-07 17:00:00+00:00         0.0
142581 2021-04-07 18:00:00+00:00         0.0
142582 2021-04-07 19:00:00+00:00         0.0
142583 2021-04-07 20:00:00+00:00         0.0

[142584 rows x 2 columns]


OSError: Cannot save file into a non-existent directory: 'data'